In [1]:
import os
import sys
import glob
import pathlib

import cv2
import torch
import matplotlib.pyplot as plt
import PIL

from PIL import Image
from torchvision.transforms import RandomCrop, RandomResizedCrop
from torchvision.transforms import RandomHorizontalFlip, ColorJitter
from torchvision.transforms import Compose, RandomGrayscale, RandomApply

In [2]:
ROOT = os.path.abspath('../')
if ROOT not in sys.path:
    sys.path.append(ROOT)

In [3]:
from datasets.transforms.torchvision import *

# 1. Load Images

In [4]:
IMG_ROOT = os.path.join(ROOT, "data/images/")
if os.path.isdir(IMG_ROOT):
    print("Sub-directories:", end=' ')
    print(*sorted(os.listdir(IMG_ROOT)), sep=', ')

Sub-directories: cars, cats, dogs, etc, flowers, places


In [5]:
jpg_files = glob.glob(os.path.join(IMG_ROOT, "**/*.jpg"), recursive=True)
print("#. images: ", len(jpg_files))

#. images:  33


In [6]:
NUM_TRIALS = 100

In [7]:
SIZE = (512, 512)

In [8]:
weak_augment = Compose([
    RandomResizedCrop(SIZE, scale=(0.2, 1.0)),
    RandomApply([ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    RandomGrayscale(p=0.2),
    RandomHorizontalFlip(0.5),
])

In [9]:
strong_augment = Compose([
    RandomResizedCrop(SIZE, scale=(0.2, 1.0)),
    RandomApply([ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    RandAugmentTv(k=5),
    RandomGrayscale(p=0.2),
    RandomHorizontalFlip(0.5),
])

In [10]:
%%time

for jpg_file in jpg_files:
    
    # Open image
    img = Image.open(jpg_file)
    
    for t in range(NUM_TRIALS + 1):
        
        write_dir, file_name = os.path.split(jpg_file)
        base_name = os.path.splitext(file_name)[0]
        write_dir = os.path.join(write_dir, base_name)
        os.makedirs(write_dir, exist_ok=True)
        
        # Weak augmentation
        weak_dir = os.path.join(write_dir, 'weak')
        weak_file = os.path.join(weak_dir, f"{t:03d}.jpg")
        os.makedirs(weak_dir, exist_ok=True)
        weak = weak_augment(img)
        weak.save(weak_file, dpi=(300, 300))
        
        # Strong augmentation
        strong_dir = os.path.join(write_dir, 'strong')
        strong_file = os.path.join(strong_dir, f"{t:03d}.jpg")
        os.makedirs(strong_dir, exist_ok=True)
        strong = strong_augment(img)
        strong.save(strong_file, dpi=(300, 300))
        
        # Random crop
        crop_dir = os.path.join(write_dir, 'crop')
        crop_file = os.path.join(crop_dir, f"{t:03d}.jpg")
        os.makedirs(crop_dir, exist_ok=True)
        crop = RandomResizedCrop(SIZE, scale=(0.5, 1.0), ratio=(1., 1.))(img)
        crop.save(crop_file, dpi=(300, 300))
    
    print('.', end='')

print('')

.................................
CPU times: user 2min 59s, sys: 477 ms, total: 3min
Wall time: 3min
